# Basic data profiling

In this notebook, I will explore the basic characteristics of the dataset and assess if it follows what the documentation describes.

## Importing data and basic cleaning

In [1]:
import json
from pathlib import Path

import numpy as np
import pandas as pd


raw_data_path = Path('../data/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx')

raw_data = pd.read_excel(raw_data_path)

raw_data

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,...,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,NaN,NaN,NaN,NaN,-1.000000,-1.000000,6-12,0
4,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-0.238095,-0.818182,-0.389967,0.407558,-0.230462,0.096774,-0.242282,-0.814433,ABOVE_12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,384,0,50th,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1921,384,0,50th,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
1922,384,0,50th,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,4-6,0
1923,384,0,50th,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,6-12,0


Cleaning columns names, by making them conform to the standard Python variable names, makes it easier to work with the data.

A standard Python variable name is composed of letters, numbers and underscores, but the first character is necessarily a letter.

In [2]:
data = raw_data.copy()

data.columns = data.columns.str.lower().str.replace(' ', '_')

There is no column name left that does not conform to the standard Python names.

In [3]:
# Return column names that do not conform to a letter followed by letters,
# numbers and underscores.
(
    data.columns
    .str.extract('([^[a-z]\w+])')
    .squeeze()
    .dropna()
)

Series([], Name: 0, dtype: object)

## Indices

According to the definitions given in the documentation, the columns *patient_visit_identifier* and *window* are not features that would make sense in any kind of predictive modeling. Given their nature, they could act together as a MultiIndex. But before doing that, they need to be validated.

### *patient_visit_identifier*

Renaming this column *id* makes it simpler to use.

In [4]:
data.rename(
    {'patient_visit_identifier': 'id'},
    axis=1,
    inplace=True,
)

data['id']

0         0
1         0
2         0
3         0
4         0
       ... 
1920    384
1921    384
1922    384
1923    384
1924    384
Name: id, Length: 1925, dtype: int64

The *id* column is apparently an ordered sequence of integers, from 0 to 384. Since there should be 5 time windows, each *id* is expected to be repeated 5 times. It can be checked against a template array.

In [5]:
id_template_array = pd.array([
    id
    for id in range(385)
    for time_window in range(5)
])

id_template_array

<IntegerArray>
[  0,   0,   0,   0,   0,   1,   1,   1,   1,   1,
 ...
 383, 383, 383, 383, 383, 384, 384, 384, 384, 384]
Length: 1925, dtype: Int64

In [6]:
data['id'].eq(id_template_array).all()

True

The equality shows the column *id* has the form we expected.

### *window*

The *window* column is composed of the 5 possible time windows.

In [7]:
data['window'].unique()

array(['0-2', '2-4', '4-6', '6-12', 'ABOVE_12'], dtype=object)

Clean its values.

In [8]:
data.loc[:, 'window'] = data.loc[:, 'window'].str.lower()

data['window'].unique()

array(['0-2', '2-4', '4-6', '6-12', 'above_12'], dtype=object)

Every patient should present each of the time windows in sequence. This can also be checked against a template.

In [9]:
windows_template_array = np.array([
    data['window'].unique()
    for patient_id in range(385)
]).ravel()

windows_template_array

array(['0-2', '2-4', '4-6', ..., '4-6', '6-12', 'above_12'], dtype=object)

In [10]:
data['window'].eq(windows_template_array).all()

True

Comparison with the proposed template confirms it has the proper format.

### MultiIndex

Since both *id* and *window* have the expected format, being both arranged in groups of 5, it's natural to use them as a MultiIndex for the DataFrame. Doing so makes it easier to understand the *DataFrame*.

In [11]:
data = data.set_index(['id', 'window'])

data

age_above65 age_percentil  gender  disease_grouping_1  \
id  window                                                            
0   0-2                 1          60th       0                 0.0   
    2-4                 1          60th       0                 0.0   
    4-6                 1          60th       0                 0.0   
    6-12                1          60th       0                 0.0   
    above_12            1          60th       0                 0.0   
...                   ...           ...     ...                 ...   
384 0-2                 0          50th       1                 0.0   
    2-4                 0          50th       1                 0.0   
    4-6                 0          50th       1                 0.0   
    6-12                0          50th       1                 0.0   
    above_12            0          50th       1                 0.0   

              disease_grouping_2  disease_grouping_3  disease_grouping_4  \
id  window                                                                 
0   0-2                      0.0                 0.0                 0.0   
    2-4                      0.0                 0.0                 0.0   
    4-6                      0.0                 0.0                 0.0   
    6-12                     0.0                 0.0                 0.0   
    above_12                 0.0                 0.0                 0.0   
...                          ...                 ...                 ...   
384 0-2                      0.0                 0.0                 0.0   
    2-4                      0.0                 0.0                 0.0   
    4-6                      0.0                 0.0                 0.0   
    6-12                     0.0                 0.0                 0.0   
    above_12                 0.0                 1.0                 0.0   

              disease_grouping_5  disease_grouping_6  htn  ...  \
id  window                                                 ...   
0   0-2                      1.0                 1.0  0.0  ...   
    2-4                      1.0                 1.0  0.0  ...   
    4-6                      1.0                 1.0  0.0  ...   
    6-12                     1.0                 1.0  0.0  ...   
    above_12                 1.0                 1.0  0.0  ...   
...                          ...                 ...  ...  ...   
384 0-2                      0.0                 0.0  0.0  ...   
    2-4                      0.0                 0.0  0.0  ...   
    4-6                      0.0                 0.0  0.0  ...   
    6-12                     0.0                 0.0  0.0  ...   
    above_12                 0.0                 0.0  0.0  ...   

              respiratory_rate_diff  temperature_diff  oxygen_saturation_diff  \
id  window                                                                      
0   0-2                   -1.000000         -1.000000               -1.000000   
    2-4                   -1.000000         -1.000000               -1.000000   
    4-6                         NaN               NaN                     NaN   
    6-12                        NaN         -1.000000               -1.000000   
    above_12               0.176471         -0.238095               -0.818182   
...                             ...               ...                     ...   
384 0-2                   -1.000000         -1.000000               -1.000000   
    2-4                   -1.000000         -1.000000               -1.000000   
    4-6                   -1.000000         -1.000000               -1.000000   
    6-12                  -1.000000         -1.000000               -1.000000   
    above_12              -0.647059         -0.547619               -0.838384   

              bloodpressure_diastolic_diff_rel  \
id  window                                       
0   0-2                              -1.000000   
    2-4                              -

## Target

The target variable is *icu*, which is either 0 (no need for ICU) or 1 (need for ICU), at a given time window.

In [12]:
data['icu'].unique()

array([0, 1])

## Features

### Groups

According to the documentation, the features can be split into four distinct groups: *demographics*, *comorbidities*, *lab tests* and *vital signs*.

In [13]:
groups = {
    'demographics': ['age_above65', 'age_percentil', 'gender'],
    'comorbidities': [
        'disease_grouping_1', 'disease_grouping_2', 'disease_grouping_3',
        'disease_grouping_4', 'disease_grouping_5', 'disease_grouping_6',
        'htn', 'immunocompromised', 'other',
    ],
    'labs': [
        'albumin_median', 'albumin_mean', 'albumin_min', 'albumin_max', 'albumin_diff',
        'be_arterial_median', 'be_arterial_mean', 'be_arterial_min', 'be_arterial_max', 'be_arterial_diff',
        'be_venous_median', 'be_venous_mean', 'be_venous_min', 'be_venous_max', 'be_venous_diff',
        'bic_arterial_median', 'bic_arterial_mean', 'bic_arterial_min', 'bic_arterial_max', 'bic_arterial_diff',
        'bic_venous_median', 'bic_venous_mean', 'bic_venous_min', 'bic_venous_max', 'bic_venous_diff',
        'billirubin_median', 'billirubin_mean', 'billirubin_min', 'billirubin_max', 'billirubin_diff',
        'blast_median', 'blast_mean', 'blast_min', 'blast_max', 'blast_diff',
        'calcium_median', 'calcium_mean', 'calcium_min', 'calcium_max', 'calcium_diff',
        'creatinin_median', 'creatinin_mean', 'creatinin_min', 'creatinin_max', 'creatinin_diff',
        'ffa_median', 'ffa_mean', 'ffa_min', 'ffa_max', 'ffa_diff',
        'ggt_median', 'ggt_mean', 'ggt_min', 'ggt_max', 'ggt_diff',
        'glucose_median', 'glucose_mean', 'glucose_min', 'glucose_max', 'glucose_diff',
        'hematocrite_median', 'hematocrite_mean', 'hematocrite_min', 'hematocrite_max', 'hematocrite_diff',
        'hemoglobin_median', 'hemoglobin_mean', 'hemoglobin_min', 'hemoglobin_max', 'hemoglobin_diff',
        'inr_median', 'inr_mean', 'inr_min', 'inr_max', 'inr_diff',
        'lactate_median', 'lactate_mean', 'lactate_min', 'lactate_max', 'lactate_diff',
        'leukocytes_median', 'leukocytes_mean', 'leukocytes_min', 'leukocytes_max', 'leukocytes_diff',
        'linfocitos_median', 'linfocitos_mean', 'linfocitos_min', 'linfocitos_max', 'linfocitos_diff',
        'neutrophiles_median', 'neutrophiles_mean', 'neutrophiles_min', 'neutrophiles_max', 'neutrophiles_diff',
        'p02_arterial_median', 'p02_arterial_mean', 'p02_arterial_min', 'p02_arterial_max', 'p02_arterial_diff',
        'p02_venous_median', 'p02_venous_mean', 'p02_venous_min', 'p02_venous_max', 'p02_venous_diff',
        'pc02_arterial_median', 'pc02_arterial_mean', 'pc02_arterial_min', 'pc02_arterial_max', 'pc02_arterial_diff',
        'pc02_venous_median', 'pc02_venous_mean', 'pc02_venous_min', 'pc02_venous_max', 'pc02_venous_diff',
        'pcr_median', 'pcr_mean', 'pcr_min', 'pcr_max', 'pcr_diff',
        'ph_arterial_median', 'ph_arterial_mean', 'ph_arterial_min', 'ph_arterial_max', 'ph_arterial_diff',
        'ph_venous_median', 'ph_venous_mean', 'ph_venous_min', 'ph_venous_max', 'ph_venous_diff',
        'platelets_median', 'platelets_mean', 'platelets_min', 'platelets_max', 'platelets_diff',
        'potassium_median', 'potassium_mean', 'potassium_min', 'potassium_max', 'potassium_diff',
        'sat02_arterial_median', 'sat02_arterial_mean', 'sat02_arterial_min', 'sat02_arterial_max', 'sat02_arterial_diff',
        'sat02_venous_median', 'sat02_venous_mean', 'sat02_venous_min', 'sat02_venous_max', 'sat02_venous_diff',
        'sodium_median', 'sodium_mean', 'sodium_min', 'sodium_max', 'sodium_diff',
        'tgo_median', 'tgo_mean', 'tgo_min', 'tgo_max', 'tgo_diff',
        'tgp_median', 'tgp_mean', 'tgp_min', 'tgp_max', 'tgp_diff',
        'ttpa_median', 'ttpa_mean', 'ttpa_min', 'ttpa_max', 'ttpa_diff',
        'urea_median', 'urea_mean', 'urea_min', 'urea_max', 'urea_diff',
        'dimer_median', 'dimer_mean', 'dimer_min', 'dimer_max', 'dimer_diff',
    ],
    'vitals': [
        'bloodpressure_diastolic_mean', 'bloodpressure_sistolic_mean', 'heart_rate_mean',
        'respiratory_rate_mean', 'temperature_mean', 'oxygen_saturation_mean',
        'bloodpressure_diastolic_median', 'bloodpressure_sistolic_median', 'heart_rate_median',
        'respiratory_rate_median', 'temperature_median', 'oxygen_saturation_median',
        'bloodpressure_diastolic_min', 'bloodpressure_sistolic_min', 'heart_rate_min',
        'respiratory_rate_min', 'temperature_min', 'oxygen_saturation_min',
        'bloodpressure_diastolic_max', 'bloodpressure_sistolic_max', 'heart_rate_max',
        'respiratory_rate_max', 'temperature_max', 'oxygen_saturation_max',
        'bloodpressure_diastolic_diff', 'bloodpressure_sistolic_diff', 'heart_rate_diff',
        'respiratory_rate_diff', 'temperature_diff', 'oxygen_saturation_diff',
        'bloodpressure_diastolic_diff_rel', 'bloodpressure_sistolic_diff_rel', 'heart_rate_diff_rel',
        'respiratory_rate_diff_rel', 'temperature_diff_rel', 'oxygen_saturation_diff_rel',
    ],
}

There are
- 3 *demographics*,
- 9 *comorbidities*,
- 36 *labs*,
- 6 *vitals*.

Each *lab* feature is actually composed of 5 distinct ones (*median*, *mean*, *min*, *max* and *diff*) and each *vital* is composed of 6 (*median*, *mean*, *min*, *max*, *diff* and *diff_rel*).

As expected, the total number of features is 3 + 9 + 36\*5 + 6\*6 = 228, which amounts to all the *DataFrame*'s columns when we add *id*, *window*, and *icu*.

### Demographics

*age_above65* should take any of the values [0, 1], meaning *no* and *yes*. *gender* also takes the values [0, 1], but it is not known which corresponds to each gender.

In [14]:
data['age_above65'].unique()

array([1, 0])

In [15]:
data['gender'].unique()

array([0, 1])

*age_percentil* should present the ten percentile intervals.

In [16]:
data['age_percentil'].unique()

array(['60th', '90th', '10th', '40th', '70th', '20th', '50th', '80th',
       '30th', 'Above 90th'], dtype=object)

This feature's values can be cleaned.

In [17]:
data.loc[:, 'age_percentil'] = (
    data.loc[:, 'age_percentil']
    .str.replace(' ', '_')
    .str.lower()
)

data['age_percentil'].unique()

array(['60th', '90th', '10th', '40th', '70th', '20th', '50th', '80th',
       '30th', 'above_90th'], dtype=object)

Since it is not expected that any of these features change whithin the dataset's time frame, their number of unique values should be 1 when considering each patient in isolation.

In [18]:
# If there's a change in any feature, nunique > 1. The sum indicates for how
# many patients the values have changed.
(
    data[groups['demographics']]
    .groupby('id')
    .agg(lambda col: col.nunique() != 1)
    .sum()
)

age_above65      0
age_percentil    0
gender           0
dtype: int64

They're all constant as expected.

### Comorbidities

Every disease group should have values in [0, 1].

In [19]:
data[groups['comorbidities']].agg(
    lambda col: col.dropna().isin([0, 1])
    .all()
)

disease_grouping_1    True
disease_grouping_2    True
disease_grouping_3    True
disease_grouping_4    True
disease_grouping_5    True
disease_grouping_6    True
htn                   True
immunocompromised     True
other                 True
dtype: bool

Checking for consistency within each patient's data.

In [20]:
(
    data[groups['comorbidities']]
    .groupby('id')
    .agg(lambda col: col.nunique() != 1)
    .sum()
)

disease_grouping_1     19
disease_grouping_2     10
disease_grouping_3     24
disease_grouping_4      4
disease_grouping_5     16
disease_grouping_6     16
htn                    45
immunocompromised       9
other                 120
dtype: int64

In this case there are several patients for whom the disease information varied. This may be due to
- Registration error.
    - Maybe when this information was being registered into the dataset, errors were made. This seems unlikely because it would mean these errors affected every comorbidity, but no demographics.
- Variation in the disease status.
    - The patients may have got better or worse for any of the disease groups.
- Variation in the disease assessment.
    - Maybe distinct health-care professionals assessed the disease status in constrasting ways at different times.

It's impossible to know for sure what is the reason for this variation. This issue will have to be dealt with later, depending on how the data processed.

### Labs

Every lab result should take a real value between -1 and 1.

In [21]:
(
    data[groups['labs']]
    .dropna()
    # Check if every absolute value is <= 1.
    .abs()
    .le(1)
    # Calling the method *all* twice checks if every single value is True.
    .all()
    .all()
)

True

### Vitals

The vitals also take values between -1 and 1.

In [22]:
(
    data[groups['vitals']]
    .dropna()
    .abs()
    .le(1)
    .all()
    .all()
)

True

## Exporting

In [23]:
data

age_above65 age_percentil  gender  disease_grouping_1  \
id  window                                                            
0   0-2                 1          60th       0                 0.0   
    2-4                 1          60th       0                 0.0   
    4-6                 1          60th       0                 0.0   
    6-12                1          60th       0                 0.0   
    above_12            1          60th       0                 0.0   
...                   ...           ...     ...                 ...   
384 0-2                 0          50th       1                 0.0   
    2-4                 0          50th       1                 0.0   
    4-6                 0          50th       1                 0.0   
    6-12                0          50th       1                 0.0   
    above_12            0          50th       1                 0.0   

              disease_grouping_2  disease_grouping_3  disease_grouping_4  \
id  window                                                                 
0   0-2                      0.0                 0.0                 0.0   
    2-4                      0.0                 0.0                 0.0   
    4-6                      0.0                 0.0                 0.0   
    6-12                     0.0                 0.0                 0.0   
    above_12                 0.0                 0.0                 0.0   
...                          ...                 ...                 ...   
384 0-2                      0.0                 0.0                 0.0   
    2-4                      0.0                 0.0                 0.0   
    4-6                      0.0                 0.0                 0.0   
    6-12                     0.0                 0.0                 0.0   
    above_12                 0.0                 1.0                 0.0   

              disease_grouping_5  disease_grouping_6  htn  ...  \
id  window                                                 ...   
0   0-2                      1.0                 1.0  0.0  ...   
    2-4                      1.0                 1.0  0.0  ...   
    4-6                      1.0                 1.0  0.0  ...   
    6-12                     1.0                 1.0  0.0  ...   
    above_12                 1.0                 1.0  0.0  ...   
...                          ...                 ...  ...  ...   
384 0-2                      0.0                 0.0  0.0  ...   
    2-4                      0.0                 0.0  0.0  ...   
    4-6                      0.0                 0.0  0.0  ...   
    6-12                     0.0                 0.0  0.0  ...   
    above_12                 0.0                 0.0  0.0  ...   

              respiratory_rate_diff  temperature_diff  oxygen_saturation_diff  \
id  window                                                                      
0   0-2                   -1.000000         -1.000000               -1.000000   
    2-4                   -1.000000         -1.000000               -1.000000   
    4-6                         NaN               NaN                     NaN   
    6-12                        NaN         -1.000000               -1.000000   
    above_12               0.176471         -0.238095               -0.818182   
...                             ...               ...                     ...   
384 0-2                   -1.000000         -1.000000               -1.000000   
    2-4                   -1.000000         -1.000000               -1.000000   
    4-6                   -1.000000         -1.000000               -1.000000   
    6-12                  -1.000000         -1.000000               -1.000000   
    above_12              -0.647059         -0.547619               -0.838384   

              bloodpressure_diastolic_diff_rel  \
id  window                                       
0   0-2                              -1.000000   
    2-4                              -

Now that the data has been validated and properly categorized, it should be exported for being used in the following notebooks.

In [24]:
import json
import pickle


data_path = Path('../data/data.pkl')
if not data_path.exists():
    with data_path.open('wb') as file:
        pickle.dump(data, file)

groups_path = Path('../data/groups.json')
if not groups_path.exists():
    with groups_path.open('w') as file:
        json.dump(groups, file, indent=2)

## Summary

This dataset mostly conforms with what is expected from the documentation. The only issue that was found is the variation of values the disease groups take for some of the patients, but it's not possible to know whether these discrepancies are expected. Care will need to be taken with this issue depending on how the data modelling is handled.

The *DataFrame* was pickled to be used in subsequent notebooks. The dictionary containing the group each feature belongs to was also exported to aid future data handling.